<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-11 12:25:19
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.32 C
-------------------
Today PnL: -48.13 K (-0.36%)
Current PnL: -22.96 L (-15.61%)
CY Booked + Current PnL: -14.69 L (-9.99%)
-------------------
Total profit:  2.02 L
Total loss:  -24.97 L
-------------------
Total Booked + Current PnL: 12.29 L (10.24%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.27%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 82.43 L (62.49%)
Deployed:  1.20 C
Current:  1.32 C
CAGR/XIRR %: 6.07%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.80,H-MC,13.90,161054.0,34854.0,11322.0,-2.45,27.62,7.03,36.59,111.0,3.08,1.24,43.59,XY24,NTT,HEALTHCARE
68,SBIN,760.30,863.00,-12.30,M-LC,3.99,217353.0,15874.0,11346.0,1.98,7.88,5.22,13.51,61.0,1.40,1.68,21.47,XY25,NTT,BANKS
36,HINDALCO,651.95,756.01,-22.38,H-LC,5.71,107464.0,3152.0,13497.0,-0.16,3.02,12.56,15.96,11.0,0.23,0.83,20.43,X5K,ATH,METALS
83,TTKPRESTIG,769.29,770.00,94.02,M-SC,10.94,83552.0,-17225.0,17320.0,0.63,-17.09,20.73,0.09,243.0,-0.99,0.65,9.77,OX40N,NTT,DURABLES
51,LALPATHLAB,2838.53,3545.00,-26.68,H-SC,8.66,219055.0,26035.0,22015.0,0.97,13.49,10.05,24.89,134.0,1.18,1.69,39.98,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ICICIPRULI,600.83,790.00,-20.77,H-MC,2.12,136425.0,637.0,42114.0,-1.05,0.47,30.87,31.48,109.0,0.02,1.05,12.64,X40,ATH,INSURANCE
25,DABUR,505.20,735.00,-9.42,H-MC,4.25,196989.0,-39.0,89669.0,-0.42,-0.02,45.52,45.49,103.0,-0.00,1.52,13.02,XY24,BTT,FMCG
58,PGHH,13388.00,18062.58,-31.21,H-MC,6.55,199035.0,-1785.0,71911.0,0.35,-0.89,36.13,34.92,82.0,-0.02,1.54,3.75,X40,ATH,FMCG


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-22.87,H-LC,1.98,205514.0,1014.0,40404.0,0.96,0.50,19.66,20.25,10.0,0.03,1.59,9.27,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.45,H-LC,2.41,168837.0,-32549.0,120161.0,0.29,-16.16,71.17,43.51,27.0,-0.27,1.30,17.53,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-8.96,H-LC,3.60,180702.0,-19608.0,79581.0,0.15,-9.79,44.04,29.94,17.0,-0.25,1.40,24.84,X200,ATH,IT
57,NESTLEIND,2268.60,2755.0,-16.62,H-LC,3.94,128279.0,-137147.0,194060.0,-0.01,-51.67,151.28,21.44,8.0,-0.71,0.99,3.74,XY25,NTT,FMCG
4,APOLLOHOSP,7098.95,8285.0,-21.61,H-LC,5.42,156541.0,364.0,25735.0,0.44,0.23,16.44,16.71,19.0,0.01,1.21,17.56,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.35,12.09,H-SC,13.17,133535.0,8238.0,50770.0,4.67,6.57,38.02,47.09,141.0,0.16,1.03,67.13,MH,ATH,POWER
71,SIEMENS,4406.90,7969.85,-2.83,H-LC,36.10,78020.0,-32152.0,121227.0,2.94,-29.18,155.38,80.85,23.0,-0.27,0.60,14.91,AR,ATH,ELECTRICAL
79,TATAMOTORS,761.77,1065.00,-24.80,M-LC,5.52,236712.0,-39811.0,149886.0,2.90,-14.40,63.32,39.81,55.0,-0.27,1.83,13.45,XY24,NTT,AUTO
20,CERA,7989.07,9475.00,-19.81,H-SC,3.07,114462.0,-29341.0,56086.0,2.47,-20.40,49.00,18.60,148.0,-0.52,0.88,26.31,OX40N,NTT,CERAMICS
68,SBIN,760.30,863.00,-12.30,M-LC,3.99,217353.0,15874.0,11346.0,1.98,7.88,5.22,13.51,61.0,1.40,1.68,21.47,XY25,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.00,103.68,M-SC,3.72,133622.0,-40937.0,40969.0,-6.49,-23.45,30.66,0.02,221.0,-1.00,1.03,15.87,OX40N,NTT,PAINTS
90,VOLTAS,1278.28,1918.49,-12.13,H-MC,8.12,184440.0,-7302.0,103342.0,-5.77,-3.81,56.03,50.08,101.0,-0.07,1.43,2.73,XY25,ATH,AC
27,DIXON,14367.00,18940.15,-36.55,H-MC,6.51,126912.0,11976.0,24608.0,-4.80,10.42,19.39,31.83,83.0,0.49,0.98,36.05,X40N,ATH,IT
52,LAOPALA,369.40,464.00,103.51,H-SC,3.17,64728.0,-36118.0,61945.0,-4.16,-35.82,95.70,25.61,144.0,-0.58,0.50,20.01,AR,NTT,CERAMICS
77,TANLA,917.30,1963.11,-29.36,H-SC,7.89,161402.0,-78013.0,350969.0,-3.67,-32.58,217.45,114.01,132.0,-0.22,1.25,48.03,AR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,94.02,M-SC,10.94,83552.0,-17225.0,17320.0,0.63,-17.09,20.73,0.09,243.0,-0.99,0.65,9.77,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,82.03,H-SC,20.58,114777.0,-27702.0,47219.0,-3.61,-19.44,41.14,13.70,163.0,-0.59,0.89,35.47,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-19.81,H-SC,3.07,114462.0,-29341.0,56086.0,2.47,-20.40,49.00,18.60,148.0,-0.52,0.88,26.31,OX40N,NTT,CERAMICS
72,SIS,477.00,528.0,2243.52,H-SC,7.06,63045.0,-18045.0,26718.0,-0.51,-22.25,42.38,10.69,155.0,-0.68,0.49,27.77,OX40N,NTT,MISC
78,TATAELXSI,7332.28,9161.0,-20.01,H-MC,2.35,79702.0,-22950.0,48554.0,-1.15,-22.36,60.92,24.94,99.0,-0.47,0.62,21.16,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.52,237331.0,-58603.0,152509.0,0.21,-19.80,64.26,31.73,2.0,-0.38,1.83,1.32,X40,BTT,IT
45,INFY,1461.46,2275.00,-22.11,H-LC,9.83,255093.0,-6508.0,152137.0,0.14,-2.49,59.64,55.67,4.0,-0.04,1.97,3.47,X40,BTT,IT
89,VBL,492.64,671.64,-10.86,H-LC,10.94,270883.0,5843.0,90448.0,1.52,2.20,33.39,36.33,5.0,0.06,2.09,15.80,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.91,H-LC,9.72,197360.0,-19159.0,112061.0,0.77,-8.85,56.78,42.91,7.0,-0.17,1.53,3.36,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.62,H-LC,3.94,128279.0,-137147.0,194060.0,-0.01,-51.67,151.28,21.44,8.0,-0.71,0.99,3.74,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,12.8,H-MC,13.9,161054.0,34854.0,11322.0,-2.45,27.62,7.03,36.59,111.0,3.08,1.24,43.59,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,12.80,H-MC,13.90,161054.0,34854.0,11322.0,-2.45,27.62,7.03,36.59,111.0,3.08,1.24,43.59,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,12.09,H-SC,13.17,133535.0,8238.0,50770.0,4.67,6.57,38.02,47.09,141.0,0.16,1.03,67.13,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,2.13,H-MC,7.66,176210.0,11540.0,56422.0,-2.12,7.01,32.02,41.28,91.0,0.20,1.36,15.55,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-7.82,H-SC,2.66,104871.0,2867.0,41057.0,0.01,2.81,39.15,43.06,123.0,0.07,0.81,20.41,X40N,ATH,MISC
68,SBIN,760.30,863.00,-12.30,M-LC,3.99,217353.0,15874.0,11346.0,1.98,7.88,5.22,13.51,61.0,1.40,1.68,21.47,XY25,NTT,BANKS


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.55,159775.0,8862.0,48444.0,-0.76,5.87,30.32,37.97,92.0,0.18,1.23,32.66,AR,ATH,AUTO
50,KPIGREEN,497.21,731.35,12.09,H-SC,13.17,133535.0,8238.0,50770.0,4.67,6.57,38.02,47.09,141.0,0.16,1.03,67.13,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,12.87,134959.0,11623.0,122138.0,-0.58,9.42,90.50,108.45,119.0,0.10,1.04,33.88,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-43.69,M-SC,4.46,95145.0,3646.0,75107.0,0.32,3.99,78.94,86.07,219.0,0.05,0.74,37.64,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-15.15,M-LC,6.67,149370.0,-1575.0,111027.0,0.78,-1.04,74.33,72.51,54.0,-0.01,1.15,4.92,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-53.72,L-SC,22.23,68678.0,-44871.0,84920.0,-0.78,-39.52,123.65,35.27,268.0,-0.53,0.53,68.09,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.67,H-SC,17.38,88769.0,-12194.0,106727.0,0.01,-12.08,120.23,93.63,149.0,-0.11,0.69,32.22,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-20.05,H-SC,15.90,90765.0,-7215.0,29081.0,0.05,-7.36,32.04,22.32,152.0,-0.25,0.70,31.64,XR,ATH,FINANCE
84,UJJIVANSFB,52.77,60.00,82.03,H-SC,20.58,114777.0,-27702.0,47219.0,-3.61,-19.44,41.14,13.70,163.0,-0.59,0.89,35.47,OX40N,NTT,BANKS
50,KPIGREEN,497.21,731.35,12.09,H-SC,13.17,133535.0,8238.0,50770.0,4.67,6.57,38.02,47.09,141.0,0.16,1.03,67.13,MH,ATH,POWER


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VBL,492.64,671.64,-10.86,H-LC,10.94,270883.0,5843.0,90448.0,1.52,2.20,33.39,36.33,5.0,0.06,2.09,15.80,X40N,ATH,FMCG
70,SHALBY,261.39,327.00,931.90,M-SC,7.99,140249.0,-40894.0,86365.0,-0.44,-22.58,61.58,25.10,232.0,-0.47,1.08,12.40,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-26.68,H-SC,8.66,219055.0,26035.0,22015.0,0.97,13.49,10.05,24.89,134.0,1.18,1.69,39.98,X40N,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-13.93,H-MC,6.55,159775.0,8862.0,48444.0,-0.76,5.87,30.32,37.97,92.0,0.18,1.23,32.66,AR,ATH,AUTO
55,MEDANTA,1087.93,1486.00,12.80,H-MC,13.90,161054.0,34854.0,11322.0,-2.45,27.62,7.03,36.59,111.0,3.08,1.24,43.59,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.49
1,25,41.16
2,50,70.85


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.49
LC    34.13
MC    26.37
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.97
X40N     15.34
X40      13.43
XY25     10.72
XR        8.98
AR        8.44
OX40N     7.56
X5K       2.23
MH        1.79
X200      1.40
SR        1.13
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.81
H-SC    24.59
H-MC    23.42
M-SC    13.45
M-LC     7.23
M-MC     2.61
L-SC     1.45
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.34,-11.07,46.77
IT,12.02,-19.93,79.83
FINANCE,8.33,-21.15,65.97
BANKS,7.65,-15.33,64.30
MISC,5.93,-19.88,88.59
PAINTS,5.65,-19.25,42.75
HEALTHCARE,5.22,3.01,21.49
ELECTRICAL,4.73,-14.72,64.40
AUTO,4.44,-17.62,68.80


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2887840.0,23
XR,1130863.0,13
AR,1116767.0,10
X40,831870.0,10
XY25,662562.0,8
X40N,622546.0,12
OX40N,480704.0,10
SR,197454.0,2
X5K,149841.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2834667.0,28
M-SC,1506039.0,17
H-LC,1397325.0,18
H-MC,1344618.0,18
M-LC,443111.0,5
M-MC,339535.0,2
L-SC,273943.0,3
L-MC,58081.0,1
L-LC,45825.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      950598.0      6
M-SC       XY24      879536.0      7
H-SC       AR        672946.0      5
           XR        532505.0      6
H-LC       X40       521950.0      5
H-MC       XY24      489286.0      5
M-MC       XY24      339535.0      2
H-LC       XY25      254317.0      3
           X40N      250201.0      5
H-MC       X40       234642.0      4
H-LC       AR        233288.0      2
H-SC       X40N      225898.0      4
H-MC       XY25      224759.0      2
H-SC       OX40N     204496.0      4
           SR        197454.0      2
M-LC       XY24      184394.0      2
L-SC       XR        183269.0      2
M-SC       AR        162089.0      2
H-MC       X40N      146447.0      3
M-SC       OX40N     136980.0      4
M-LC       X5K       136344.0      1
M-SC       XY25      126315.0      1
           XR        125841.0      2
H-MC       XR        120140.0      1
M-LC       XR        111027.0      1
L-SC       OX40N      90674.0      1
H-LC       X200       79581.0      1
M-SC       X40        75278.0      1
L-MC       XR         58081.0      1
H-SC       MH         50770.0      1
H-MC       OX40N      48554.0      1
           AR         48444.0      1
L-LC       XY25       45825.0      1
H-LC       XY24       44491.0      1
H-MC       MH         32346.0      1
H-LC       X5K        13497.0      1
M-LC       XY25       11346.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
